## Set Root Directory and Out Directory

In [ ]:
import os
import time

ROOT_DIR = os.path.abspath('')
OUT_DIR = os.path.join(ROOT_DIR, 'out')

if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)

## Load WebDriver for Chrome
https://sites.google.com/a/chromium.org/chromedriver/downloads

In [ ]:
DRIVER = os.path.join(ROOT_DIR, 'chromedriver')

In [ ]:
from selenium import webdriver

driver = webdriver.Chrome(DRIVER)

driver.implicitly_wait(5)

window = {}

window['main'] = driver.window_handles[-1]

## Open Page

In [ ]:
ECAMPUS = 'https://ecampus.ut.ac.kr'

In [ ]:

driver.get(ECAMPUS)

## Load Authentication Info

In [ ]:
import json

SECRET_JSON = os.path.join(ROOT_DIR, 'secrets.json')

In [ ]:
with open(SECRET_JSON) as f:
    secrets = json.load(f)

login_id = secrets["ID"]
login_pw = secrets["PW"]

## Login

In [ ]:
def logout():
    btn_logout = driver.find_element_by_id('btn_logout')
    btn_logout.click()

def login(_id, _pw):
    try:
        logout()
    except:
        pass

    # Enter Info
    input_id = driver.find_element_by_id('id')
    input_pw = driver.find_element_by_id('pass')

    input_id.send_keys(_id)
    input_pw.send_keys(_pw)

    # Login
    driver.execute_script('login_proc()')

In [ ]:
time.sleep(5)
login(login_id, login_pw)

login_id, login_pw = (None, None)
time.sleep(10)

In [ ]:
panel = driver.find_element_by_id('selfInfoAfter')
lecture_list = panel.find_element_by_class_name('lecInfo')

print(lecture_list)

lectures = lecture_list.find_elements_by_xpath("//a[contains(., '2020')]")

print(lectures)


## Enter The Lecture

In [ ]:
sample_lecture = lectures[0]

sample_lecture.click()

You can enter the 'lecture room' with path below.
It will lead you to the lecture room of the last lecture you entered...

In [ ]:
lecture_room_url = "https://ecampus.ut.ac.kr/lms/class/courseSchedule/doListView.dunet"

driver.get(lecture_room_url)
time.sleep(2)

## Get courses list


In [ ]:
# not_progressed_list = driver.find_elements_by_xpath("//td[contains(., 'not progressed')]")
current_courses_link = driver.find_elements_by_xpath("//a[contains(., 'Lecture view')]")
current_courses = [course_link.find_element_by_xpath("../..") for course_link in current_courses_link]

current_courses_data =  []
titles = []
links = []
mins = []
for course in current_courses:
    datas = course.find_elements_by_tag_name('td')
    title = datas[1].text
    lecture_time = datas[2].text
    period = datas[3].text
    status = datas[4].text
    link = datas[5].find_element_by_class_name('lectureWindow')
    # link = datas[5]

    print(title, lecture_time, period, status, link)
    print()
    if status != "Complete":
        titles.append(title)
        links.append(link)
        mins.append(int(lecture_time[:-6]))

print(titles)

## Check Study Time and Open Lecture Window

In [ ]:
import tqdm

print("{} courses.".format(len(links)))
seconds = [minute * 60 for minute in mins]
for sec,  title, link in tqdm.tqdm(zip(seconds, titles, links)):
    print("{} for {}minutes...".format(title, sec//60))
    link.click()
    window_lecture = driver.window_handles[-1]
    time.sleep(sec + 100)

    driver.switch_to_window(window_lecture)
    driver.close()
    print("Course End")

print("Finished.")

In [ ]:


# sample_link = links[-1]
# sample_course = sample_link.find_element_by_xpath("../..")
# tds = sample_course.find_elements_by_tag_name('td')
# # sample_lecture_time = abcs[-1].find_element_by_name('td')[2][:-7]
# print(tds[1].text)

#TODO: "not progressed and lecture VIEW'

# sample_link.click()
# window['lecture'] = driver.window_handles[-1]
# time.sleep(5)

In [ ]:
window['lecture']